In [4]:
import requests
import pandas as pd
import json
api_key = 'xPzSUT5g3ThZfCWYnNpkLMtbU8OpDaAk6dDczzAg'

In [5]:
senators_response = requests.get(f"https://api.propublica.org/congress/v1/members/senate/GA/current.json", headers={'X-API-Key': api_key}).json()
senators_response
congress_info = {
    "name" : [],
    "id" : [],
    "gender" : [],
    "party" : [],
    "role" : [],
    "seniority" : [],
    "next_election" : [],
    "district" : []
}

congress_info["name"].append(senators_response["results"][0]["name"])
congress_info["id"].append(senators_response["results"][0]["id"])
congress_info["gender"].append(senators_response["results"][0]["gender"])
congress_info["party"].append(senators_response["results"][0]["party"])
congress_info["role"].append(senators_response["results"][0]["role"])
congress_info["seniority"].append(senators_response["results"][0]["seniority"])
congress_info["next_election"].append(senators_response["results"][0]["next_election"])
congress_info["district"].append("0")
    
congress_info["name"].append(senators_response["results"][1]["name"])
congress_info["id"].append(senators_response["results"][1]["id"])
congress_info["gender"].append(senators_response["results"][1]["gender"])
congress_info["party"].append(senators_response["results"][1]["party"])
congress_info["role"].append(senators_response["results"][1]["role"])
congress_info["seniority"].append(senators_response["results"][1]["seniority"])
congress_info["next_election"].append(senators_response["results"][1]["next_election"])
congress_info["district"].append("0")

In [6]:
districts =["1", "2","3","4","5","6","7","8","9","10","11","12","13","14"]

for district in districts:
    house_response = requests.get(f"https://api.propublica.org/congress/v1/members/house/GA/{district}/current.json", headers={'X-API-Key': api_key}).json()
    congress_info["name"].append(house_response["results"][0]["name"])
    congress_info["id"].append(house_response["results"][0]["id"])
    congress_info["gender"].append(house_response["results"][0]["gender"])
    congress_info["party"].append(house_response["results"][0]["party"])
    congress_info["role"].append(house_response["results"][0]["role"])
    congress_info["seniority"].append(house_response["results"][0]["seniority"])
    congress_info["next_election"].append(house_response["results"][0]["next_election"])
    congress_info["district"].append(district)

congress_info

{'name': ['David Perdue',
  'Kelly Loeffler',
  'Buddy Carter',
  'Sanford D. Bishop Jr.',
  'A. Drew Ferguson',
  'Hank Johnson',
  'John Lewis',
  'Lucy McBath',
  'Rob Woodall',
  'Austin Scott',
  'Doug Collins',
  'Jody Hice',
  'Barry Loudermilk',
  'Rick Allen',
  'David Scott',
  'Tom Graves'],
 'id': ['P000612',
  'L000594',
  'C001103',
  'B000490',
  'F000465',
  'J000288',
  'L000287',
  'M001208',
  'W000810',
  'S001189',
  'C001093',
  'H001071',
  'L000583',
  'A000372',
  'S001157',
  'G000560'],
 'gender': ['M',
  'F',
  'M',
  'M',
  'M',
  'M',
  'M',
  'F',
  'M',
  'M',
  'M',
  'M',
  'M',
  'M',
  'M',
  'M'],
 'party': ['R',
  'R',
  'R',
  'D',
  'R',
  'D',
  'D',
  'D',
  'R',
  'R',
  'R',
  'R',
  'R',
  'R',
  'D',
  'R'],
 'role': ['Senator, 2nd Class',
  'Senator, 3rd Class',
  'Representative',
  'Representative',
  'Representative',
  'Representative',
  'Representative',
  'Representative',
  'Representative',
  'Representative',
  'Representative',


In [8]:
voter_pop = ["0", "0", "539387","473245","602082","491317","502193","567076","642070","530981","609141",
"567614","583126","523257","564612","691975"]
congress_info["district_voters"] = voter_pop

In [9]:
congress_df = pd.DataFrame(congress_info)
congress_df.rename(columns = {"id" : "member_id"}, inplace = True)
congress_df.head(10)

,name,member_id,gender,party,role,seniority,next_election,district,district_voters
0,David Perdue,P000612,M,R,"Senator, 2nd Class",1,2020,0,0
1,Kelly Loeffler,L000594,F,R,"Senator, 3rd Class",1,2022,0,0
2,Buddy Carter,C001103,M,R,Representative,6,2020,1,539387
3,Sanford D. Bishop Jr.,B000490,M,D,Representative,28,2020,2,473245
4,A. Drew Ferguson,F000465,M,R,Representative,4,2020,3,602082
5,Hank Johnson,J000288,M,D,Representative,14,2020,4,491317
6,John Lewis,L000287,M,D,Representative,34,2020,5,502193
7,Lucy McBath,M001208,F,D,Representative,2,2020,6,567076
8,Rob Woodall,W000810,M,R,Representative,10,2020,7,642070
9,Austin Scott,S001189,M,R,Representative,10,2020,8,530981


In [10]:
votes_dict = {
    "roll_call" : [],
    "bill_name" : [],
    "bill_info" : [],
    "member_id" : [],
    "result" : [],
    "date" : [],
    "total_votes" : [],
    "position" : []
    }
for item in congress_df["member_id"]:
    response = requests.get(f"https://api.propublica.org/congress/v1/members/{item}/votes.json", headers={'X-API-Key': api_key}).json()
    for item in response["results"][0]["votes"]:
        votes_dict["roll_call"].append(item["roll_call"])
        votes_dict["bill_name"].append(item["description"])
        votes_dict["bill_info"].append(item["bill"])
        votes_dict["member_id"].append(item["member_id"])
        votes_dict["result"].append(item["result"])
        votes_dict["date"].append(item["date"])
        votes_dict["total_votes"].append(item["total"])
        votes_dict["position"].append(item["position"])


In [11]:
#create dataframe from dictionary
votes_df = pd.DataFrame(votes_dict)

#NOTES: bill_info and total_votes columns are dictionaries. Havent figured out how we want to sort that data yet
#       roll_call seems to be the instance of voting, with some bills having been voted on multiple times. Need to look into further
#votes_df.info()
#print(votes_df["bill_info"][2])

votes_df = pd.concat([votes_df.drop(['total_votes'], axis=1), votes_df['total_votes'].apply(pd.Series)], axis=1)
votes_df = pd.concat([votes_df.drop(['bill_info'], axis=1), votes_df['bill_info'].apply(pd.Series)], axis=1)
votes_df.head()
#votes_df.info()

,roll_call,bill_name,member_id,result,date,position,yes,no,present,not_voting,bill_id,number,api_uri,title,latest_action,sponsor_id,bill_uri
0,130,"Russell Vought, of Virginia, to be Director of...",P000612,Cloture Motion Agreed to,2020-07-02,Yes,47,44,0,9,pn1726-116,PN1726,NaN,None,None,NaN,NaN
1,129,To withdraw all United States Armed Forces fro...,P000612,Motion to Table Agreed to,2020-07-01,Yes,60,33,0,7,s4049-116,S. 4049,NaN,An original bill to authorize appropriations f...,Cloture motion on the measure presented in Sen...,I000024,https://api.propublica.org/congress/v1/116/bil...
2,128,An original bill to authorize appropriations f...,P000612,Motion to Proceed Agreed to,2020-06-29,Yes,89,4,0,7,s4049-116,S.4049,NaN,An original bill to authorize appropriations f...,Cloture motion on the measure presented in Sen...,I000024,https://api.propublica.org/congress/v1/116/bil...
3,127,An original bill to authorize appropriations f...,P000612,Cloture on the Motion to Proceed Agreed to,2020-06-25,Yes,90,7,0,3,s4049-116,S.4049,NaN,An original bill to authorize appropriations f...,Cloture motion on the measure presented in Sen...,I000024,https://api.propublica.org/congress/v1/116/bil...
4,126,A bill to improve and reform policing practice...,P000612,Cloture on the Motion to Proceed Rejected,2020-06-24,Yes,55,45,0,0,s3985-116,S.3985,NaN,A bill to improve and reform policing practice...,Motion by Senator McConnell to reconsider the ...,S001184,https://api.propublica.org/congress/v1/116/bil...


### 

In [12]:
sponsored_dict = {
    "congress": [],
    "title" : [],
    "short_title" : [],
    "bill_id" : [],
    "bill_type" : [],
    "sponsor_id" : [],
    "introduced_date" : [],
    "primary_subject" : [],
    "active" : [],
    "last_vote": [],
    "house_passage": [],
    "senate_passage": [],
    "enacted": [],
    "vetoed": [],
    "cosponsors": [],
    "summary": [],
    "summary_short": [],
    "latest_major_action_date": [],
    "latest_major_action": [],
    "bill_status" : []
    }
bill_type = ["introduced", "updated", "active", "passed", "enacted", "vetoed"]
for member in congress_df["member_id"]:
    for bill in bill_type:
        bill_response = requests.get(f"https://api.propublica.org/congress/v1/members/{member}/bills/{bill}.json", headers={'X-API-Key': api_key}).json()
        for item in bill_response["results"][0]["bills"]:
            sponsored_dict["congress"].append(item["congress"])
            sponsored_dict["title"].append(item["title"])
            sponsored_dict["short_title"].append(item["short_title"])
            sponsored_dict["bill_id"].append(item["bill_id"])
            sponsored_dict["bill_type"].append(item["bill_type"])
            sponsored_dict["sponsor_id"].append(item["sponsor_id"])
            sponsored_dict["introduced_date"].append(item["introduced_date"])
            sponsored_dict["primary_subject"].append(item["primary_subject"])
            sponsored_dict["active"].append(item["active"])
            sponsored_dict["last_vote"].append(item["last_vote"])
            sponsored_dict["house_passage"].append(item["house_passage"])
            sponsored_dict["senate_passage"].append(item["senate_passage"])
            sponsored_dict["enacted"].append(item["enacted"])
            sponsored_dict["vetoed"].append(item["vetoed"])
            sponsored_dict["cosponsors"].append(item["cosponsors"])
            sponsored_dict["summary"].append(item["summary"])
            sponsored_dict["summary_short"].append(item["summary_short"])
            sponsored_dict["latest_major_action_date"].append(item["latest_major_action_date"])
            sponsored_dict["latest_major_action"].append(item["latest_major_action"])
            sponsored_dict["bill_status"].append(bill)
            



KeyError: 'results'

In [3]:
sponsored_df = pd.DataFrame(sponsored_dict)
sponsored_df[0]
#sponsored_df.info()
sponsored_df["primary_subject"].unique()

NameError: name 'sponsored_dict' is not defined

In [8]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [9]:
engine = create_engine('sqlite:///static/data/reps.sqlite', echo=True)
sqlite_connection = engine.connect()

2020-07-11 15:07:14,938 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-11 15:07:14,945 INFO sqlalchemy.engine.base.Engine ()
2020-07-11 15:07:14,949 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-11 15:07:14,950 INFO sqlalchemy.engine.base.Engine ()


In [10]:
sqlite_table = "Representatives"
congress_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2020-07-11 15:07:33,718 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Representatives")
2020-07-11 15:07:33,720 INFO sqlalchemy.engine.base.Engine ()
2020-07-11 15:07:33,724 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Representatives")
2020-07-11 15:07:33,726 INFO sqlalchemy.engine.base.Engine ()
2020-07-11 15:07:33,728 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-11 15:07:33,730 INFO sqlalchemy.engine.base.Engine ()
2020-07-11 15:07:33,733 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Representatives")
2020-07-11 15:07:33,735 INFO sqlalchemy.engine.base.Engine ()
2020-07-11 15:07:33,739 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Representatives' AND type = 'table'
2020-07-11 15:07:33,740 INFO sqlalchemy.engine.base.Engine ()
2020-07-11 15:07:33,744 INFO sqlalchemy.engine.base.Engine PRAG

In [14]:
engine.execute("SELECT * FROM Representatives").fetchall()

2020-07-10 18:18:51,215 INFO sqlalchemy.engine.base.Engine SELECT * FROM Representatives
2020-07-10 18:18:51,217 INFO sqlalchemy.engine.base.Engine ()


[(0, 'David Perdue', 'P000612', 'M', 'R', 'Senator, 2nd Class', '1', '2020', '0'),
 (1, 'Kelly Loeffler', 'L000594', 'F', 'R', 'Senator, 3rd Class', '1', '2022', '0'),
 (2, 'Buddy Carter', 'C001103', 'M', 'R', 'Representative', '6', '2020', '1'),
 (3, 'Sanford D. Bishop Jr.', 'B000490', 'M', 'D', 'Representative', '28', '2020', '2'),
 (4, 'A. Drew Ferguson', 'F000465', 'M', 'R', 'Representative', '4', '2020', '3'),
 (5, 'Hank Johnson', 'J000288', 'M', 'D', 'Representative', '14', '2020', '4'),
 (6, 'John Lewis', 'L000287', 'M', 'D', 'Representative', '34', '2020', '5'),
 (7, 'Lucy McBath', 'M001208', 'F', 'D', 'Representative', '2', '2020', '6'),
 (8, 'Rob Woodall', 'W000810', 'M', 'R', 'Representative', '10', '2020', '7'),
 (9, 'Austin Scott', 'S001189', 'M', 'R', 'Representative', '10', '2020', '8'),
 (10, 'Doug Collins', 'C001093', 'M', 'R', 'Representative', '8', '2020', '9'),
 (11, 'Jody Hice', 'H001071', 'M', 'R', 'Representative', '6', '2020', '10'),
 (12, 'Barry Loudermilk', 'L

In [15]:
sqlite_table = "Votes"
votes_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2020-07-10 18:18:56,324 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Votes")
2020-07-10 18:18:56,325 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 18:18:56,327 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Votes")
2020-07-10 18:18:56,328 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 18:18:56,334 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Votes" (
	"index" BIGINT, 
	roll_call TEXT, 
	bill_name TEXT, 
	member_id TEXT, 
	result TEXT, 
	date TEXT, 
	position TEXT, 
	yes BIGINT, 
	no BIGINT, 
	present BIGINT, 
	not_voting BIGINT, 
	bill_id TEXT, 
	number TEXT, 
	api_uri FLOAT, 
	title TEXT, 
	latest_action TEXT, 
	sponsor_id TEXT, 
	bill_uri TEXT
)


2020-07-10 18:18:56,335 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 18:18:56,345 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-10 18:18:56,347 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Votes_index" ON "Votes" ("index")
2020-07-10 18:18:56,349 INFO sqlalchemy.engine.base.Engine ()
2020-0

In [16]:
engine.execute("SELECT * FROM Votes").keys()

2020-07-10 18:19:03,165 INFO sqlalchemy.engine.base.Engine SELECT * FROM Votes
2020-07-10 18:19:03,167 INFO sqlalchemy.engine.base.Engine ()


['index',
 'roll_call',
 'bill_name',
 'member_id',
 'result',
 'date',
 'position',
 'yes',
 'no',
 'present',
 'not_voting',
 'bill_id',
 'number',
 'api_uri',
 'title',
 'latest_action',
 'sponsor_id',
 'bill_uri']

In [17]:
sqlite_table = "Sponsored"
sponsored_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2020-07-10 18:19:07,848 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Sponsored")
2020-07-10 18:19:07,850 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 18:19:07,852 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Sponsored")
2020-07-10 18:19:07,853 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 18:19:07,860 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Sponsored" (
	"index" BIGINT, 
	congress TEXT, 
	title TEXT, 
	short_title TEXT, 
	bill_id TEXT, 
	bill_type TEXT, 
	sponsor_id TEXT, 
	introduced_date TEXT, 
	primary_subject TEXT, 
	active BOOLEAN, 
	last_vote TEXT, 
	house_passage TEXT, 
	senate_passage TEXT, 
	enacted TEXT, 
	vetoed TEXT, 
	cosponsors BIGINT, 
	summary TEXT, 
	summary_short TEXT, 
	latest_major_action_date TEXT, 
	latest_major_action TEXT, 
	bill_status TEXT, 
	CHECK (active IN (0, 1))
)


2020-07-10 18:19:07,861 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 18:19:07,869 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-10 18:19:07

In [18]:
engine.execute("SELECT * FROM Sponsored").keys()

2020-07-10 18:19:15,554 INFO sqlalchemy.engine.base.Engine SELECT * FROM Sponsored
2020-07-10 18:19:15,557 INFO sqlalchemy.engine.base.Engine ()


['index',
 'congress',
 'title',
 'short_title',
 'bill_id',
 'bill_type',
 'sponsor_id',
 'introduced_date',
 'primary_subject',
 'active',
 'last_vote',
 'house_passage',
 'senate_passage',
 'enacted',
 'vetoed',
 'cosponsors',
 'summary',
 'summary_short',
 'latest_major_action_date',
 'latest_major_action',
 'bill_status']

In [19]:
sqlite_connection.close()